In [3]:
import sys, os

module_path = os.path.abspath(os.getcwd() + '/../../..')
print(module_path)

os.environ["OPENAI_API_KEY"] = ""

sys.path.append(module_path)

/home/jupyter/green.ai


In [6]:
from deepeval.dataset import Golden
from deepeval.test_case import LLMTestCase
from openai import AsyncOpenAI, OpenAI
import re

from benchmarq.utility import VLLMServerSingleton
from benchmarq.utility import Evaluator
from benchmarq.experiment import Experiment

from dotenv import load_dotenv

load_dotenv()

csv_files = ["100",
             "500",
             "1000",
             "2000",
             "3000",
             "4000",
             "5000",
             "10000",
             "20000", ]

server = VLLMServerSingleton()

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

async_client = AsyncOpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

server.start_server(model="EleutherAI/pythia-1.4b")

class Test(Evaluator):
    def __init__(self):
        super().__init__()

    async def async_evaluate_consumption(self, input: Golden):
        return await async_client.completions.create(model="EleutherAI/pythia-1.4b", prompt=f"{input.context} {input.input}", max_tokens=50)

    def evaluate_test_case(self, input: Golden) -> LLMTestCase:
        output = client.completions.create(model="EleutherAI/pythia-1.4b", prompt=f"{input.context} {input.input}", max_tokens=50).choices[0].text

        return LLMTestCase(input=input.input, expected_output=input.expected_output, actual_output=output, context=input.context, retrieval_context=input.retrieval_context)

for dataset in csv_files:
    experiment = Experiment(
        subquestion_id="input_size",
        id=dataset,
        dataset_name=dataset,
        subquestion_path="experiments/context_window/input_size/tests.json",
        name=dataset,
        description=f"A test to see if energy readings change based on input size. Current dataset size: {dataset}",
        settings=Test())
    a = await experiment.run()

server.stop_server()

Server is already running.


IndexError: list index out of range